In [ ]:
import os
import shutil
import sh
import subprocess
from pyspark.sql.functions import *

In [ ]:
directory = '/user/marekdragan/New_Tweets/'
file = '*.json'
path = directory + file

In [ ]:
tweets_df = spark.read.json(path)

In [ ]:
tweets_df.printSchema()

In [65]:
tweets_df_l1 = tweets_df.select("id", "lang", "created_at", "user.screen_name", \
                                #"user.name",\
                                "user.location", "retweet_count", "text")
#You MAY run out of memory with caching - use with caution
#tweets_df_l1.cache()
type(tweets_df_l1)

pyspark.sql.dataframe.DataFrame

In [66]:
tweets_df_l1.show(10)

+------------------+----+--------------------+---------------+------------------+-------------+--------------------+
|                id|lang|          created_at|    screen_name|          location|retweet_count|                text|
+------------------+----+--------------------+---------------+------------------+-------------+--------------------+
|851505580489547776|  en|Mon Apr 10 18:42:...|   USPressWorld|        Etats-Unis|            0|Illinois Judge Sh...|
|851505581613559808|  en|Mon Apr 10 18:42:...|         TRIYZ_|              null|            0|RT @Smooky_Montan...|
|851505581806338048|  en|Mon Apr 10 18:42:...| KristyBerridge| Cairns, Australia|            0|RT @LIONPubs: Sav...|
|851505584213983235|  en|Mon Apr 10 18:42:...|   kylah_hudson|              hell|            0|RT @SLAYLONIE: yo...|
|851505583995924480|  en|Mon Apr 10 18:42:...|       Suntimes|       Chicago, IL|            0|Dancer-actress Ch...|
|851505583765245955|  en|Mon Apr 10 18:42:...|    5tweetstogo|  

In [86]:
tweets_df_l2 = tweets_df_l1.where("lower(text) like '%uchicago%' \
                        or lower(text) like '%hyde park%'\
                        or (text) like '%Booth%'\
                        or lower(text) like '%regenstein%' \
                        or lower(text) like '%the midway%' \
                        or (text) like '%u of c%' \
                        or (text) like '%Nobel%' \
                        or lower(text) like '%gleacher%'")
tweets_df_l2.count()

7939

In [101]:
tweets_df_l2 = tweets_df_l1.where("lower(text) like '%uchicago%' \
                        or lower(text) like '%hyde park%'\
                        or (text) like '%Booth%'\
                        or lower(text) like '%regenstein%' \
                        or lower(text) like '%the midway%' \
                        or (text) like '%u of c%' \
                        or (text) like '%Nobel%' \
                        or (text) like '%University of Chicago%' \
                        or lower(text) like '%gleacher%'")
tweets_df_l2.count()

32321

In [ ]:
tweets_df_l2 = tweets_df_l1.where("lower(text) like '%northwestern%' \
                        or lower(text) like '%evanston%'\
                        or (text) like '%Kellogg%'\
                        or (text) like '%Nobel%' \
                        
tweets_df_l2.count()

In [102]:
tweets_df_l2.select("text").show(5, truncate = False)

+--------------------------------------------------------------------------------------------------------------------------------------------+
|text                                                                                                                                        |
+--------------------------------------------------------------------------------------------------------------------------------------------+
|RT @MEF_Psych_Dept: This Thursday, Rob Booth will give a talk to the Cognitive Neuroscience group at Istanbul University Faculty of Med…    |
|Information Session presented by Brown University, Columbia, Rice, and University of Chicago. RSVP at... https://t.co/JmOi2yyhbg            |
|A Fulbright scholar at the University of Chicago went home to visit his parents in Pakistan for the winter holid... https://t.co/koYh4zh63l |
|#Bulls #Card: 1996-1997 Michael Jordan Bowman's Best Chicago Bulls #80 YuChicago#3352 https://t.co/avpubEOiMN #Cards https://t.co/xpRhjfO2J2|

In [103]:
tweets_df_l3 = tweets_df_l2.withColumn("text_clean", regexp_replace("text", "[\\r\\n\\t]", " "))
#withColumn("text_clean", trim(lower(col("text_clean"))))

In [104]:
tweets_df_l3.select("text_clean").show(5, truncate = False)

+--------------------------------------------------------------------------------------------------------------------------------------------+
|text_clean                                                                                                                                  |
+--------------------------------------------------------------------------------------------------------------------------------------------+
|RT @MEF_Psych_Dept: This Thursday, Rob Booth will give a talk to the Cognitive Neuroscience group at Istanbul University Faculty of Med…    |
|Information Session presented by Brown University, Columbia, Rice, and University of Chicago. RSVP at... https://t.co/JmOi2yyhbg            |
|A Fulbright scholar at the University of Chicago went home to visit his parents in Pakistan for the winter holid... https://t.co/koYh4zh63l |
|#Bulls #Card: 1996-1997 Michael Jordan Bowman's Best Chicago Bulls #80 YuChicago#3352 https://t.co/avpubEOiMN #Cards https://t.co/xpRhjfO2J2|

In [105]:
tweets_df_l3.count()

32321

In [106]:
tweets_df_l4 = tweets_df_l3.drop('text')

In [107]:
tweets_df_l4.show(5)

+------------------+----+--------------------+---------------+--------------------+-------------+--------------------+
|                id|lang|          created_at|    screen_name|            location|retweet_count|          text_clean|
+------------------+----+--------------------+---------------+--------------------+-------------+--------------------+
|851508016549679104|  en|Mon Apr 10 18:51:...|        ccerenk|                null|            0|RT @MEF_Psych_Dep...|
|851509914891558912|  en|Mon Apr 10 18:59:...| NHS_Counseling|                null|            0|Information Sessi...|
|851512442337316864|  en|Mon Apr 10 19:09:...|     thus_spake|England, Great Br...|            0|A Fulbright schol...|
|851512674668036096|  en|Mon Apr 10 19:10:...|  Jonelle_Donis|                null|            0|#Bulls #Card: 199...|
|851513886641618944|  en|Mon Apr 10 19:15:...|BauerJournalism|   Chicago, Illinois|            0|A man, 18, was sh...|
+------------------+----+--------------------+--

In [108]:
# Save data
twit_out_hdfs = "hdfs:///user/jfnavarr/twitter_out_3"
twit_out_linx_dir = "/home/jfnavarr/temp/"
twit_out_linx_file = "twitter_out_3.txt"
twit_out_linx = twit_out_linx_dir + twit_out_linx_file

In [109]:
try:
    sh.hdfs('dfs','-rmr',twit_out_hdfs)
except:
    print(twit_out_hdfs+' *** Does not exist ***')

In [110]:
#tweets_df_l4.write.format("com.databricks.spark.csv").option("header", "false").option("delimiter", "\t").save(twit_out_hdfs)
tweets_df_l4.write.format("com.databricks.spark.csv").option("header", "true").option("delimiter", "\t").save(twit_out_hdfs)

In [111]:
try:
    print(sh.hdfs('dfs','-ls',twit_out_hdfs))
except:
    print(twit_out_hdfs+' *** Does not exist ***')

Found 654 items
-rw-r--r--   3 jfnavarr jfnavarr          0 2017-06-01 08:44 hdfs:///user/jfnavarr/twitter_out_2/_SUCCESS
-rw-r--r--   3 jfnavarr jfnavarr       1068 2017-06-01 08:44 hdfs:///user/jfnavarr/twitter_out_2/part-r-00000-3e1d7085-ae62-4a62-a56f-267460de6289.csv
-rw-r--r--   3 jfnavarr jfnavarr       3300 2017-06-01 08:44 hdfs:///user/jfnavarr/twitter_out_2/part-r-00001-3e1d7085-ae62-4a62-a56f-267460de6289.csv
-rw-r--r--   3 jfnavarr jfnavarr       1176 2017-06-01 08:44 hdfs:///user/jfnavarr/twitter_out_2/part-r-00002-3e1d7085-ae62-4a62-a56f-267460de6289.csv
-rw-r--r--   3 jfnavarr jfnavarr       1363 2017-06-01 08:43 hdfs:///user/jfnavarr/twitter_out_2/part-r-00003-3e1d7085-ae62-4a62-a56f-267460de6289.csv
-rw-r--r--   3 jfnavarr jfnavarr        260 2017-06-01 08:44 hdfs:///user/jfnavarr/twitter_out_2/part-r-00004-3e1d7085-ae62-4a62-a56f-267460de6289.csv
-rw-r--r--   3 jfnavarr jfnavarr       4335 2017-06-01 08:44 hdfs:///user/jfnavarr/twitter_out_2/part-r-00005-3e1d7085-ae62

In [112]:
def run_cmd(args_list):
    print('Running system command: {0}'.format(' '.join(args_list)))
    proc = subprocess.Popen(args_list, stdout=subprocess.PIPE,
            stderr=subprocess.PIPE)
    (output, errors) = proc.communicate()
    if proc.returncode:
        raise RuntimeError(
            'Error running command: %s. Return code: %d, Error: %s' % (join(args_list), proc.returncode, errors))
    return (output, errors)

In [113]:
(out, errors)= run_cmd(['hadoop', 'fs', '-getmerge', twit_out_hdfs, twit_out_linx])

Running system command: hadoop fs -getmerge hdfs:///user/jfnavarr/twitter_out_2 /home/jfnavarr/temp/twitter_out_2.txt


In [114]:
os.listdir(twit_out_linx_dir)

['.twitter_out.txt.crc',
 'twitter_out_2.txt',
 'twitter_out.txt',
 '.twitter_out_2.txt.crc']